# 👁️ Mécanismes d'Attention - Module 6

Ce notebook explore en détail les mécanismes d'attention qui sont au cœur des Transformers.

## 🎯 Objectifs
- Comprendre le concept d'attention
- Implémenter Self-Attention avec TensorFlow
- Explorer Multi-Head Attention
- Visualiser les matrices d'attention
- Comparer avec les approches RNN/LSTM

In [ ]:
# Installation des dépendances
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print(f"🔧 TensorFlow version: {tf.__version__}")
print(f"🔧 NumPy version: {np.__version__}")
print(f"📊 GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 1. 🧠 Intuition de l'Attention

L'attention permet à un modèle de se concentrer sur les parties importantes de l'input lors de chaque prédiction.

In [ ]:
def visualize_attention_concept():
    """
    Visualise le concept d'attention sur une phrase exemple
    """
    sentence = ["Le", "chat", "noir", "mange", "des", "croquettes"]
    
    # Simulation d'une matrice d'attention
    attention_weights = np.array([
        [0.1, 0.7, 0.1, 0.05, 0.025, 0.025],  # "Le" fait attention à "chat"
        [0.1, 0.5, 0.3, 0.05, 0.025, 0.025],  # "chat" à lui-même et "noir"
        [0.05, 0.6, 0.25, 0.05, 0.025, 0.025], # "noir" fait attention à "chat"
        [0.05, 0.3, 0.1, 0.4, 0.1, 0.05],     # "mange" équilibré
        [0.05, 0.1, 0.05, 0.3, 0.3, 0.2],     # "des" vers "croquettes"
        [0.05, 0.1, 0.05, 0.2, 0.2, 0.4]      # "croquettes" vers lui-même
    ])
    
    # Création de la heatmap
    fig = go.Figure(data=go.Heatmap(
        z=attention_weights,
        x=sentence,
        y=sentence,
        colorscale='Reds',
        text=np.round(attention_weights, 3),
        texttemplate="%{text}",
        textfont={"size":10},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title="🎯 Matrice d'Attention : Qui regarde qui ?",
        xaxis_title="Mots de la phrase (clés)",
        yaxis_title="Mots de la phrase (requêtes)",
        width=600,
        height=500
    )
    
    fig.show()
    
    print("💡 Interprétation :")
    print("- Les couleurs intenses montrent où chaque mot 'fait attention'")
    print("- 'Le' fait attention à 'chat' (relation déterminant-nom)")
    print("- 'noir' fait attention à 'chat' (adjectif-nom)")
    print("- 'mange' regarde à la fois le sujet et l'objet")
    
visualize_attention_concept()

## 2. 🔍 Implémentation de l'Attention Scaled Dot-Product

La formule fondamentale : **Attention(Q,K,V) = softmax(QK^T/√d_k)V**

In [ ]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
    """
    Implémentation de l'attention Scaled Dot-Product
    """
    
    def __init__(self, temperature=None, **kwargs):
        super(ScaledDotProductAttention, self).__init__(**kwargs)
        self.temperature = temperature
        
    def call(self, queries, keys, values, mask=None, training=None):
        """
        Args:
            queries: [batch_size, seq_len, d_k]
            keys: [batch_size, seq_len, d_k]
            values: [batch_size, seq_len, d_v]
            mask: Masque optionnel
        
        Returns:
            output: [batch_size, seq_len, d_v]
            attention_weights: [batch_size, seq_len, seq_len]
        """
        # Étape 1: Calcul des scores QK^T
        scores = tf.matmul(queries, keys, transpose_b=True)
        
        # Étape 2: Normalisation par √d_k
        d_k = tf.cast(tf.shape(keys)[-1], tf.float32)
        if self.temperature is not None:
            scores = scores / self.temperature
        else:
            scores = scores / tf.math.sqrt(d_k)
        
        # Étape 3: Application du masque (si fourni)
        if mask is not None:
            scores += (mask * -1e9)
        
        # Étape 4: Softmax pour obtenir les poids d'attention
        attention_weights = tf.nn.softmax(scores, axis=-1)
        
        # Étape 5: Application des poids aux valeurs
        output = tf.matmul(attention_weights, values)
        
        return output, attention_weights

# Test de l'implémentation
def test_attention():
    print("🧪 Test de l'Attention Scaled Dot-Product")
    print("=" * 50)
    
    # Paramètres
    batch_size = 2
    seq_len = 5
    d_model = 64
    
    # Création de données d'exemple
    tf.random.set_seed(42)
    queries = tf.random.normal([batch_size, seq_len, d_model])
    keys = tf.random.normal([batch_size, seq_len, d_model])
    values = tf.random.normal([batch_size, seq_len, d_model])
    
    # Instanciation de la couche d'attention
    attention_layer = ScaledDotProductAttention()
    
    # Calcul de l'attention
    output, attention_weights = attention_layer(queries, keys, values)
    
    print(f"📐 Shape des queries: {queries.shape}")
    print(f"📐 Shape des keys: {keys.shape}")
    print(f"📐 Shape des values: {values.shape}")
    print(f"✅ Shape de l'output: {output.shape}")
    print(f"📊 Shape des poids d'attention: {attention_weights.shape}")
    
    # Vérification que les poids somment à 1
    sum_weights = tf.reduce_sum(attention_weights, axis=-1)
    print(f"🔍 Somme des poids (doit être ~1.0): {tf.reduce_mean(sum_weights):.6f}")
    
    return output, attention_weights

output, attention_weights = test_attention()

## 3. 🎨 Visualisation des Poids d'Attention

In [ ]:
def visualize_attention_weights(attention_weights, sentence=None):
    """
    Visualise les poids d'attention avec Plotly
    """
    # Prendre le premier exemple du batch
    weights = attention_weights[0].numpy()
    
    if sentence is None:
        sentence = [f"Token_{i}" for i in range(weights.shape[0])]
    
    # Création de la heatmap interactive
    fig = go.Figure(data=go.Heatmap(
        z=weights,
        x=sentence,
        y=sentence,
        colorscale='Viridis',
        text=np.round(weights, 3),
        texttemplate="%{text}",
        textfont={"size":8},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title="🔥 Heatmap des Poids d'Attention",
        xaxis_title="Keys (ce qui est regardé)",
        yaxis_title="Queries (qui regarde)",
        width=600,
        height=500
    )
    
    fig.show()

# Visualisation avec notre exemple
sentence = ["Je", "mange", "une", "pomme", "rouge"]
visualize_attention_weights(attention_weights, sentence)

## 4. 🧠 Multi-Head Attention

Le Multi-Head Attention permet d'avoir plusieurs "perspectives" d'attention en parallèle.

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    """
    Implémentation du Multi-Head Attention
    """
    
    def __init__(self, d_model, num_heads, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        
        # Couches linéaires pour Q, K, V
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        
        # Couche de sortie
        self.dense = tf.keras.layers.Dense(d_model)
        
        # Couche d'attention
        self.attention = ScaledDotProductAttention()
        
    def split_heads(self, x, batch_size):
        """
        Divise la dernière dimension en (num_heads, depth)
        Transpose pour avoir shape (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask=None, training=None):
        batch_size = tf.shape(q)[0]
        
        # 1. Transformation linéaire et division en têtes
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # 2. Attention scaled dot-product
        scaled_attention, attention_weights = self.attention(
            q, k, v, mask=mask, training=training
        )
        
        # 3. Concaténation des têtes
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, 
                                    (batch_size, -1, self.d_model))
        
        # 4. Projection finale
        output = self.dense(concat_attention)
        
        return output, attention_weights

# Test du Multi-Head Attention
def test_multihead_attention():
    print("🧪 Test du Multi-Head Attention")
    print("=" * 40)
    
    # Paramètres
    d_model = 512
    num_heads = 8
    batch_size = 2
    seq_len = 10
    
    # Données d'exemple
    tf.random.set_seed(42)
    x = tf.random.normal([batch_size, seq_len, d_model])
    
    # Création de la couche Multi-Head Attention
    mha = MultiHeadAttention(d_model, num_heads)
    
    # Self-attention (Q, K, V sont identiques)
    output, attention_weights = mha(x, x, x)
    
    print(f"📐 Input shape: {x.shape}")
    print(f"✅ Output shape: {output.shape}")
    print(f"📊 Attention weights shape: {attention_weights.shape}")
    print(f"🧠 Nombre de têtes: {num_heads}")
    print(f"🔧 Dimension par tête: {d_model // num_heads}")
    
    return output, attention_weights

mha_output, mha_attention_weights = test_multihead_attention()

## 5. 🎭 Visualisation Multi-Head

In [ ]:
def visualize_multihead_attention(attention_weights, sentence=None, max_heads=4):
    """
    Visualise les différentes têtes d'attention
    """
    # Prendre le premier exemple du batch
    weights = attention_weights[0].numpy()  # Shape: [num_heads, seq_len, seq_len]
    
    if sentence is None:
        seq_len = weights.shape[1]
        sentence = [f"T{i}" for i in range(seq_len)]
    
    num_heads = min(weights.shape[0], max_heads)
    
    # Création des sous-graphiques
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"Tête {i+1}" for i in range(num_heads)],
        vertical_spacing=0.08,
        horizontal_spacing=0.05
    )
    
    for i in range(num_heads):
        row = (i // 2) + 1
        col = (i % 2) + 1
        
        fig.add_trace(
            go.Heatmap(
                z=weights[i],
                x=sentence,
                y=sentence,
                colorscale='Viridis',
                showscale=(i == 0),
                text=np.round(weights[i], 2),
                texttemplate="%{text}",
                textfont={"size":8}
            ),
            row=row, col=col
        )
    
    fig.update_layout(
        title="🧠 Multi-Head Attention - Différentes Perspectives",
        height=800
    )
    
    fig.show()
    
    # Analyse des patterns
    print("🔍 Analyse des patterns d'attention:")
    for i in range(num_heads):
        # Calcul de la concentration (entropie inverse)
        entropy = -np.sum(weights[i] * np.log(weights[i] + 1e-10), axis=-1)
        avg_entropy = np.mean(entropy)
        
        if avg_entropy < 1.0:
            pattern = "Très focalisée"
        elif avg_entropy < 1.5:
            pattern = "Modérément focalisée"
        else:
            pattern = "Diffuse"
            
        print(f"  Tête {i+1}: {pattern} (entropie: {avg_entropy:.2f})")

# Test avec une phrase exemple
sentence_example = ["The", "cat", "sat", "on", "the", "mat"]

# Créer un exemple plus petit pour la visualisation
d_model = 512
num_heads = 8
seq_len = len(sentence_example)
batch_size = 1

tf.random.set_seed(123)
x_small = tf.random.normal([batch_size, seq_len, d_model])
mha_small = MultiHeadAttention(d_model, num_heads)
_, small_attention_weights = mha_small(x_small, x_small, x_small)

visualize_multihead_attention(small_attention_weights, sentence_example)

## 6. 🔄 Self-Attention vs Cross-Attention

In [ ]:
def demonstrate_attention_types():
    """
    Démontre la différence entre Self-Attention et Cross-Attention
    """
    print("🔄 Démonstration: Self-Attention vs Cross-Attention")
    print("=" * 55)
    
    # Paramètres
    d_model = 256
    num_heads = 4
    batch_size = 1
    
    # Séquences d'exemple
    source_len = 5  # "Hello how are you ?"
    target_len = 4  # "Bonjour comment ça"
    
    tf.random.set_seed(42)
    source_seq = tf.random.normal([batch_size, source_len, d_model])
    target_seq = tf.random.normal([batch_size, target_len, d_model])
    
    mha = MultiHeadAttention(d_model, num_heads)
    
    # 1. Self-Attention sur la séquence source
    print("1️⃣ Self-Attention (source regarde source):")
    self_output, self_weights = mha(source_seq, source_seq, source_seq)
    print(f"   Input: {source_seq.shape}")
    print(f"   Output: {self_output.shape}")
    print(f"   Attention weights: {self_weights.shape}")
    
    # 2. Cross-Attention (target regarde source)
    print("\n2️⃣ Cross-Attention (target regarde source):")
    cross_output, cross_weights = mha(
        v=source_seq,  # Values viennent de la source
        k=source_seq,  # Keys viennent de la source
        q=target_seq   # Queries viennent de la target
    )
    print(f"   Queries (target): {target_seq.shape}")
    print(f"   Keys/Values (source): {source_seq.shape}")
    print(f"   Output: {cross_output.shape}")
    print(f"   Attention weights: {cross_weights.shape}")
    
    return self_weights, cross_weights

self_attn_weights, cross_attn_weights = demonstrate_attention_types()

## 7. 🎯 Application Pratique: Analyse de Sentiment

In [ ]:
class AttentionSentimentAnalyzer(tf.keras.Model):
    """
    Modèle d'analyse de sentiment utilisant l'attention
    """
    
    def __init__(self, vocab_size, d_model=128, num_heads=4, max_length=100, **kwargs):
        super(AttentionSentimentAnalyzer, self).__init__(**kwargs)
        
        self.d_model = d_model
        self.max_length = max_length
        
        # Couches d'embedding
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_encoding = self.positional_encoding(max_length, d_model)
        
        # Couche d'attention
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Feed Forward
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(d_model * 4, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        # Classification
        self.global_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(1, activation='sigmoid')
    
    def positional_encoding(self, position, d_model):
        """
        Crée l'encodage positionnel
        """
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model
        )
        
        # Sinus pour les positions paires
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        # Cosinus pour les positions impaires
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)
    
    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        return pos * angle_rates
    
    def call(self, x, training=None, return_attention=False):
        seq_len = tf.shape(x)[1]
        
        # Embedding + Positional Encoding
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        
        # Self-Attention
        attn_output, attention_weights = self.mha(x, x, x, training=training)
        x = self.layernorm1(x + attn_output)
        
        # Feed Forward
        ffn_output = self.ffn(x)
        x = self.layernorm2(x + ffn_output)
        
        # Classification
        x = self.global_pool(x)
        x = self.dropout(x, training=training)
        output = self.classifier(x)
        
        if return_attention:
            return output, attention_weights
        return output

# Test du modèle
def test_sentiment_model():
    print("🎭 Test du Modèle de Sentiment avec Attention")
    print("=" * 50)
    
    # Paramètres
    vocab_size = 10000
    d_model = 128
    num_heads = 4
    max_length = 50
    
    # Données d'exemple
    batch_size = 4
    seq_len = 20
    
    # Séquences aléatoires (simulent des phrases tokenisées)
    x = tf.random.uniform([batch_size, seq_len], maxval=vocab_size, dtype=tf.int32)
    
    # Création du modèle
    model = AttentionSentimentAnalyzer(vocab_size, d_model, num_heads, max_length)
    
    # Prédiction
    predictions, attention_weights = model(x, return_attention=True)
    
    print(f"📐 Input shape: {x.shape}")
    print(f"✅ Predictions shape: {predictions.shape}")
    print(f"📊 Attention weights shape: {attention_weights.shape}")
    print(f"🎯 Exemple de prédictions: {predictions.numpy().flatten()[:4]}")
    
    # Statistiques du modèle
    total_params = sum([tf.size(v).numpy() for v in model.trainable_variables])
    print(f"🔧 Nombre total de paramètres: {total_params:,}")
    
    return model, attention_weights

sentiment_model, sentiment_attention = test_sentiment_model()

## 8. 📊 Comparaison RNN vs Attention

In [ ]:
def compare_rnn_vs_attention():
    """
    Compare les performances et caractéristiques RNN vs Attention
    """
    print("⚖️ Comparaison RNN vs Attention")
    print("=" * 40)
    
    # Simulation de benchmarks
    metrics = {
        'Architecture': ['RNN/LSTM', 'Self-Attention', 'Multi-Head Attention'],
        'Parallélisation': ['❌ Séquentiel', '✅ Parallèle', '✅ Parallèle'],
        'Complexité': ['O(n)', 'O(n²)', 'O(n²)'],
        'Mémoire à long terme': ['Limitée', '✅ Illimitée', '✅ Illimitée'],
        'Vitesse d\'entraînement': ['Lent', 'Rapide', 'Très rapide'],
        'Performance (BLEU)': [25.4, 32.1, 35.8],
        'Paramètres (M)': [2.5, 8.2, 12.1]
    }
    
    # Création du tableau comparatif
    import pandas as pd
    df = pd.DataFrame(metrics)
    
    # Affichage stylé
    print(df.to_string(index=False))
    
    # Graphique de performance
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=metrics['Paramètres (M)'],
        y=metrics['Performance (BLEU)'],
        mode='markers+lines+text',
        text=metrics['Architecture'],
        textposition="top center",
        marker=dict(size=15, color=['red', 'blue', 'green']),
        line=dict(width=2)
    ))
    
    fig.update_layout(
        title="📈 Performance vs Complexité",
        xaxis_title="Nombre de Paramètres (Millions)",
        yaxis_title="Score BLEU (Traduction)",
        width=800,
        height=500
    )
    
    fig.show()
    
    print("\n💡 Observations clés:")
    print("• L'attention permet la parallélisation → entraînement plus rapide")
    print("• Pas de limite de distance → meilleure mémoire à long terme")
    print("• Plus de paramètres mais performance supérieure")
    print("• Complexité O(n²) peut être problématique pour très longues séquences")

compare_rnn_vs_attention()

## 9. 🎮 Attention Interactive

In [ ]:
def interactive_attention_demo():
    """
    Démonstration interactive de l'attention
    """
    print("🎮 Démonstration Interactive de l'Attention")
    print("=" * 45)
    
    # Phrases d'exemple avec différents patterns
    examples = [
        "The cat sat on the mat",
        "Despite the rain, the game continued",
        "She loves him but he doesn't care",
        "The quick brown fox jumps over the lazy dog"
    ]
    
    for i, sentence in enumerate(examples):
        print(f"\n{i+1}. Phrase: '{sentence}'")
        
        # Tokenisation simple
        tokens = sentence.lower().split()
        vocab = {word: idx for idx, word in enumerate(set(tokens))}
        token_ids = [vocab[word] for word in tokens]
        
        # Création d'embeddings aléatoires mais cohérents
        tf.random.set_seed(42 + i)  # Seed différent pour chaque phrase
        embeddings = tf.random.normal([len(tokens), 64])
        
        # Calcul de l'attention
        attention_layer = ScaledDotProductAttention()
        _, attention_weights = attention_layer(
            embeddings[None, ...], 
            embeddings[None, ...], 
            embeddings[None, ...]
        )
        
        # Analyse des patterns
        weights = attention_weights[0].numpy()
        
        # Trouvez les paires de mots avec la plus forte attention
        max_attention_pairs = []
        for j in range(len(tokens)):
            max_idx = np.argmax(weights[j])
            if max_idx != j:  # Éviter l'auto-attention
                max_attention_pairs.append(
                    (tokens[j], tokens[max_idx], weights[j, max_idx])
                )
        
        # Affichage des top 3 relations
        max_attention_pairs.sort(key=lambda x: x[2], reverse=True)
        print("   Relations d'attention les plus fortes:")
        for word1, word2, strength in max_attention_pairs[:3]:
            print(f"   • '{word1}' → '{word2}' ({strength:.3f})")
        
        # Visualisation rapide (première phrase seulement)
        if i == 0:
            visualize_attention_weights(attention_weights, tokens)

interactive_attention_demo()

## 10. 🔮 Perspectives et Applications Avancées

In [ ]:
def advanced_attention_applications():
    """
    Présente les applications avancées de l'attention
    """
    print("🔮 Applications Avancées de l'Attention")
    print("=" * 45)
    
    applications = {
        "🌍 Traduction Automatique": {
            "Description": "Cross-attention entre langues source et cible",
            "Avantage": "Alignement automatique des mots",
            "Exemple": "'Je mange' → 'I eat' avec attention sur correspondances"
        },
        "📝 Résumé de Texte": {
            "Description": "Attention sur phrases et concepts importants",
            "Avantage": "Identification automatique des éléments clés",
            "Exemple": "Article de 1000 mots → résumé de 100 mots"
        },
        "❓ Question-Réponse": {
            "Description": "Attention entre question et passages du texte",
            "Avantage": "Localisation précise des réponses",
            "Exemple": "'Qui est le président?' → attention sur noms propres"
        },
        "🎨 Génération d'Images": {
            "Description": "Attention entre description et régions de l'image",
            "Avantage": "Génération cohérente et contrôlée",
            "Exemple": "'Un chat noir sur un tapis rouge' → attention spatiale"
        },
        "🗣️ Reconnaissance Vocale": {
            "Description": "Attention entre signal audio et transcription",
            "Avantage": "Alignement temporel automatique",
            "Exemple": "Spectrogramme → texte avec correspondances temporelles"
        }
    }
    
    for app_name, details in applications.items():
        print(f"\n{app_name}:")
        print(f"  📋 {details['Description']}")
        print(f"  ✨ {details['Avantage']}")
        print(f"  💡 {details['Exemple']}")
    
    # Tendances futures
    print("\n🚀 Tendances Futures:")
    future_trends = [
        "Sparse Attention: Réduction de la complexité O(n²)",
        "Linear Attention: Attention en O(n)",
        "Cross-Modal Attention: Vision + Texte + Audio",
        "Hierarchical Attention: Attention multi-niveaux",
        "Adaptive Attention: Longueur de contexte dynamique"
    ]
    
    for trend in future_trends:
        print(f"  • {trend}")

advanced_attention_applications()

## 🎯 Conclusion et Récapitulatif

### Ce que vous avez appris :

✅ **Concept d'Attention** : Mécanisme permettant de se concentrer sur les parties importantes  
✅ **Scaled Dot-Product Attention** : Formule mathématique et implémentation  
✅ **Multi-Head Attention** : Plusieurs perspectives d'attention en parallèle  
✅ **Self vs Cross Attention** : Différences et cas d'usage  
✅ **Applications Pratiques** : Sentiment, traduction, génération  
✅ **Avantages vs RNN** : Parallélisation et mémoire à long terme

### Prochaines étapes :
- **Module 6.2** : Architecture complète des Transformers
- **Module 7** : BERT et GPT
- **Projets pratiques** : Implémentation de vos propres modèles

### 🛠️ Exercices Pratiques Suggérés :
1. Modifiez les paramètres d'attention (température, nombre de têtes)
2. Implémentez différents types de masques
3. Créez un modèle de traduction simple
4. Analysez les patterns d'attention sur vos propres textes

In [ ]:
# 🎊 Félicitations ! Vous maîtrisez maintenant les mécanismes d'attention !
print("""🎊 FÉLICITATIONS ! 🎊

Vous venez de maîtriser un des concepts les plus importants du NLP moderne !

🧠 L'attention est utilisée dans :
  • GPT (ChatGPT, GPT-4)
  • BERT (Google Search)
  • T5 (Google Translate)
  • DALL-E (génération d'images)
  • Et bien d'autres...

🚀 Continuez vers l'architecture Transformer complète !
""")